# Import

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 9.4 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load dataset & model

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

In [6]:
x_normal = df[df['Label'] == 0].reset_index(drop=True)

In [7]:
x_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54797 entries, 0 to 54796
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      54797 non-null  float64
 1   Tot Fwd Pkts       54797 non-null  float64
 2   Tot Bwd Pkts       54797 non-null  float64
 3   TotLen Fwd Pkts    54797 non-null  float64
 4   TotLen Bwd Pkts    54797 non-null  float64
 5   Fwd Pkt Len Max    54797 non-null  float64
 6   Fwd Pkt Len Min    54797 non-null  float64
 7   Fwd Pkt Len Mean   54797 non-null  float64
 8   Fwd Pkt Len Std    54797 non-null  float64
 9   Bwd Pkt Len Max    54797 non-null  float64
 10  Bwd Pkt Len Min    54797 non-null  float64
 11  Bwd Pkt Len Mean   54797 non-null  float64
 12  Bwd Pkt Len Std    54797 non-null  float64
 13  Flow Byts/s        54797 non-null  float64
 14  Flow Pkts/s        54797 non-null  float64
 15  Flow IAT Mean      54797 non-null  float64
 16  Flow IAT Std       547

In [8]:
x_normal['Label'].value_counts()

0    54797
Name: Label, dtype: int64

In [9]:
df = df.drop(columns=['Label'])

In [10]:
x_normal = x_normal.drop(columns=['Label'])

In [11]:
feature_list = x_normal.columns

# SHAP

In [12]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100, random_state = 42))

In [13]:
normal_1500 = x_normal.head(1500)

In [14]:
shap_values = explainer.shap_values(normal_1500)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [15]:
data_df = pd.DataFrame(shap_values[0])

In [ ]:
data_df.to_csv('/content/drive/MyDrive/1500_samples_[0].csv', index=False)

In [16]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,-0.044187,0.013770,0.000000,-0.004049,0.000000,-0.008802,0.000000,-0.014032,-0.010526,0.038564,...,0.000000,0.0,0.000000,-0.017420,0.000000,0.000000,-0.016809,0.018498,0.024251,0.000000
1,0.000000,0.000000,0.003570,-0.010793,-0.006418,-0.002193,0.000000,-0.005705,0.000000,-0.003380,...,0.000000,0.0,0.005172,0.000000,0.006109,0.000000,0.007509,-0.011611,0.008820,0.011227
2,-0.001246,-0.003812,0.006438,0.000000,0.000000,-0.008148,-0.005095,-0.018360,0.008413,0.004801,...,-0.003721,0.0,0.005441,0.002102,0.000000,-0.011588,0.006370,0.002538,0.004108,0.000000
3,-0.004588,0.000000,0.003159,0.003865,0.000000,0.008838,-0.004413,-0.008484,-0.003987,-0.001338,...,0.009813,0.0,0.000000,-0.008573,0.000000,0.005783,-0.003417,-0.002453,0.006396,0.000000
4,-0.007091,0.008490,0.003210,0.005818,0.000000,-0.015870,0.000000,-0.003857,-0.015554,0.000000,...,-0.001845,0.0,0.002949,0.000000,-0.003723,0.003384,0.000000,-0.003419,0.000000,0.009838


In [17]:
data_dict = {}
for idx in range(0, 1500):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Normal sample ' + str(idx)] = row_values

In [18]:
print(data_dict)

{'Normal sample 0': ['ACK Flag Cnt', 'Bwd IAT Max', 'Bwd Pkt Len Max', 'Idle Max', 'Bwd IAT Tot', 'Bwd Header Len', 'Idle Std', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Bwd Pkts'], 'Normal sample 1': ['Init Bwd Win Byts', 'Fwd IAT Max', 'Idle Min', 'Fwd IAT Tot', 'Idle Max', 'Fwd IAT Std', 'Idle Mean', 'Bwd Header Len', 'Fwd IAT Mean', 'Active Max'], 'Normal sample 2': ['Init Bwd Win Byts', 'Pkt Len Var', 'Fwd Pkt Len Std', 'Subflow Bwd Byts', 'Tot Bwd Pkts', 'Idle Mean', 'Fwd IAT Std', 'Active Mean', 'Flow IAT Min', 'Bwd Pkt Len Max'], 'Normal sample 3': ['Init Bwd Win Byts', 'Fwd Act Data Pkts', 'Flow IAT Mean', 'Fwd Pkt Len Max', 'Idle Max', 'Bwd IAT Min', 'Bwd Header Len', 'Active Min', 'Bwd IAT Mean', 'Subflow Bwd Byts'], 'Normal sample 4': ['SYN Flag Cnt', 'Idle Min', 'Tot Fwd Pkts', 'ACK Flag Cnt', 'Fwd IAT Tot', 'TotLen Fwd Pkts', 'Fwd IAT Mean', 'Bwd IAT Min', 'Bwd IAT Mean', 'Flow Byts/s'], 'Normal sample 5': ['Init Bwd Win Byts', 'Fwd IAT Max', 'Idle Mean', 'Fwd Header Len', 

In [19]:
len(data_dict)

1500

# New whitelist: 38 top features with over 10k freq each

In [20]:
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, Normal sample 0 to Normal sample 1499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Feature 1   1500 non-null   object
 1   Feature 2   1500 non-null   object
 2   Feature 3   1500 non-null   object
 3   Feature 4   1500 non-null   object
 4   Feature 5   1500 non-null   object
 5   Feature 6   1500 non-null   object
 6   Feature 7   1500 non-null   object
 7   Feature 8   1500 non-null   object
 8   Feature 9   1500 non-null   object
 9   Feature 10  1500 non-null   object
dtypes: object(10)
memory usage: 128.9+ KB


In [22]:
df.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10
Normal sample 0,ACK Flag Cnt,Bwd IAT Max,Bwd Pkt Len Max,Idle Max,Bwd IAT Tot,Bwd Header Len,Idle Std,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Bwd Pkts
Normal sample 1,Init Bwd Win Byts,Fwd IAT Max,Idle Min,Fwd IAT Tot,Idle Max,Fwd IAT Std,Idle Mean,Bwd Header Len,Fwd IAT Mean,Active Max
Normal sample 2,Init Bwd Win Byts,Pkt Len Var,Fwd Pkt Len Std,Subflow Bwd Byts,Tot Bwd Pkts,Idle Mean,Fwd IAT Std,Active Mean,Flow IAT Min,Bwd Pkt Len Max
Normal sample 3,Init Bwd Win Byts,Fwd Act Data Pkts,Flow IAT Mean,Fwd Pkt Len Max,Idle Max,Bwd IAT Min,Bwd Header Len,Active Min,Bwd IAT Mean,Subflow Bwd Byts
Normal sample 4,SYN Flag Cnt,Idle Min,Tot Fwd Pkts,ACK Flag Cnt,Fwd IAT Tot,TotLen Fwd Pkts,Fwd IAT Mean,Bwd IAT Min,Bwd IAT Mean,Flow Byts/s


In [ ]:
df.to_csv('/content/drive/MyDrive/MLP_top10_of_each_normalfile.csv')

In [23]:
freq = {}
for column in df.columns:
  for feature in df[column]:
    if feature in freq:
      continue
    else:
      count = sum(df[column_2] == feature for column_2 in df.columns)
      freq[feature] = count.sum()
      print(freq[feature])

146
786
293
245
341
364
184
265
276
89
413
15
174
3
531
449
181
432
169
203
205
273
399
395
296
323
164
308
172
170
202
372
546
191
382
210
143
158
273
167
175
143
224
186
74
184
198
373
150
231
140
275
176
200
122
135
208
231
207
147
170
136
3
2
2


In [24]:
len(freq)

65

In [25]:
freq_df = pd.DataFrame(list(freq.items()), columns=['Feature', 'Frequency'])
freq_df.head()

,Feature,Frequency
0,ACK Flag Cnt,146
1,Init Bwd Win Byts,786
2,SYN Flag Cnt,293
3,Down/Up Ratio,245
4,Bwd IAT Tot,341


In [26]:
freq_df.sort_values(by=['Frequency'], ascending=False, inplace=True)
freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 1 to 64
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Feature    65 non-null     object
 1   Frequency  65 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [28]:
freq_df.head(38)

,Feature,Frequency
1,Init Bwd Win Byts,786
32,Idle Max,546
14,Bwd Pkt Len Max,531
15,Idle Mean,449
17,Tot Fwd Pkts,432
10,Idle Min,413
22,Bwd Pkt Len Std,399
23,Tot Bwd Pkts,395
34,Bwd Pkt Len Mean,382
47,TotLen Fwd Pkts,373


In [31]:
freq_df.to_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv', index=False)

# Detect normal samples

In [48]:
whitelist = freq_df.iloc[:, 0][:51].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean']


In [49]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:752


In [50]:
whitelist = freq_df.iloc[:, 0][:59].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean', 'ACK Flag Cnt', 'Flow IAT Max', 'Active Min', 'Pkt Len Mean', 'Fwd Pkt Len Mean', 'Flow IAT Std', 'Fwd Seg Size Avg', 'PSH Flag Cnt']


In [51]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:86


## False positive of XAI on normal samples of trainset:
top 38: 1308/1500

top 24: 1482/1500